In [71]:
# libraries

from datetime import datetime
import os
import glob
import requests 
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [18]:
link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSc_2y5N0I67wDU38DjDh35IZSIS30rQf7_NYZhtYYGU1jJYT6_kDx4YpF-qw0LSlGsBYP8pqM_a1Pd/pubhtml#'
req = requests.get(link)
soup = BeautifulSoup(req.content, "html.parser")

In [24]:
# thead = soup.find_all('thead')[0]
# head = thead.find_all('tr')

# print(thead)
# print(head)`

In [27]:
print(len(soup.find_all('tbody')))

14


In [29]:
tbody = soup.find_all('tbody')[0]
body = tbody.find_all('tr')

# print(tbody)
# print(body)

In [39]:
head_row = [i.text for i in body[0].find_all('td')]
head_row

['Patient Number',
 'State Patient Number',
 'Date Announced',
 'Age Bracket',
 'Gender',
 'Detected City',
 'Detected District',
 'Detected State',
 'Current Status',
 'Notes',
 'Contracted from which Patient (Suspected)',
 'Nationality',
 'Status Change Date',
 'Source_1',
 'Source_2',
 'Source_3',
 'Backup Notes',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [45]:
contents = []

for i in range(len(body)):
    contents.append([i.text for i in body[i].find_all('td')])

In [49]:
p_df = pd.DataFrame(contents[2:len(contents)], columns=head_row)
p_df.head()

,Patient Number,State Patient Number,Date Announced,Age Bracket,Gender,Detected City,Detected District,Detected State,Current Status,Notes,...,Source_2,Source_3,Backup Notes,,,,,,,
0,1,KL-TS-P1,30/01/2020,20,F,Thrissur,Thrissur,Kerala,Recovered,Travelled from Wuhan,...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,,,,,,,,None
1,2,KL-AL-P1,02/02/2020,,,Alappuzha,Alappuzha,Kerala,Recovered,Travelled from Wuhan,...,https://weather.com/en-IN/india/news/news/2020...,,Student from Wuhan,,,,,,,
2,3,KL-KS-P1,03/02/2020,,,Kasaragod,Kasaragod,Kerala,Recovered,Travelled from Wuhan,...,https://twitter.com/ANI/status/122422148580539...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,,,,,,,
3,4,DL-P1,02/03/2020,45,M,East Delhi (Mayur Vihar),East Delhi,Delhi,Recovered,"Travelled from Austria, Italy",...,https://economictimes.indiatimes.com/news/poli...,,Travel history to Italy and Austria,,,,,,,
4,5,TS-P1,02/03/2020,24,M,Hyderabad,Hyderabad,Telangana,Recovered,,...,https://www.indiatoday.in/india/story/coronavi...,https://www.thehindu.com/news/national/coronav...,"Travel history to Dubai, Singapore contact",,,,,,,


In [51]:
p_df.shape

(2072, 24)

In [57]:
p_df.columns

Index(['Patient Number', 'State Patient Number', 'Date Announced',
       'Age Bracket', 'Gender', 'Detected City', 'Detected District',
       'Detected State', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Status Change Date', 'Source_1', 'Source_2', 'Source_3',
       'Backup Notes', '', '', '', '', '', '', ''],
      dtype='object')

In [58]:
p_df = p_df[['Patient Number', 'State Patient Number', 'Date Announced',
       'Age Bracket', 'Gender', 'Detected City', 'Detected District',
       'Detected State', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Status Change Date', 'Source_1', 'Source_2', 'Source_3',
       'Backup Notes']]

In [59]:
p_df.isna().sum()

Patient Number                               0
State Patient Number                         0
Date Announced                               0
Age Bracket                                  0
Gender                                       0
Detected City                                0
Detected District                            0
Detected State                               0
Current Status                               0
Notes                                        0
Contracted from which Patient (Suspected)    0
Nationality                                  0
Status Change Date                           0
Source_1                                     0
Source_2                                     0
Source_3                                     0
Backup Notes                                 0
dtype: int64

(2072, 17)

In [79]:
# no. of empty strings in each column
# ===================================

print(p_df.shape)

for i in p_df.columns:
    print(i, '\t', p_df[p_df[i]==''].shape[0])

(2072, 17)
Patient Number 	 0
State Patient Number 	 0
Date Announced 	 0
Age Bracket 	 0
Gender 	 0
Detected City 	 0
Detected District 	 0
Detected State 	 0
Current Status 	 0
Notes 	 0
Contracted from which Patient (Suspected) 	 0
Nationality 	 0
Status Change Date 	 0
Source_1 	 0
Source_2 	 0
Source_3 	 0
Backup Notes 	 0


In [80]:
# replacing empty strings with np.nan
# ==================================-

print(p_df.shape)

p_df = p_df.replace(r'', np.nan, regex=True)
p_df.isna().sum()

(2072, 17)


Patient Number                                767
State Patient Number                         1783
Date Announced                                876
Age Bracket                                  1536
Gender                                       1494
Detected City                                1402
Detected District                            1034
Detected State                                876
Current Status                                876
Notes                                         884
Contracted from which Patient (Suspected)    1823
Nationality                                  1656
Status Change Date                            879
Source_1                                      899
Source_2                                     1485
Source_3                                     1931
Backup Notes                                 1712
dtype: int64

In [81]:
# droping empty rows (row with just row number but without patient entry
# ===================

print(p_df.shape)
p_df.dropna(subset=['Detected State'], inplace=True)
print(p_df.shape)

(2072, 17)
(1196, 17)


# Saving Data

In [77]:
p_df.sample(5)

,Patient Number,State Patient Number,Date Announced,Age Bracket,Gender,Detected City,Detected District,Detected State,Current Status,Notes,Contracted from which Patient (Suspected),Nationality,Status Change Date,Source_1,Source_2,Source_3,Backup Notes
711,712,KL-TV-R6,26/03/2020,36,M,NaN,Malappuram,Kerala,Hospitalized,"Travelled from Oman, Oman Air WY 215",NaN,NaN,26/03/2020,https://twitter.com/xpresskerala/status/124315...,https://www.facebook.com/malappuramcollector/p...,NaN,NaN
1642,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
294,295,NaN,21/03/2020,26,M,Gandhinagar,Gandhinagar,Gujarat,Hospitalized,Travelled from Dubai,NaN,India,21/03/2020,https://twitter.com/ANI/status/124129257358444...,https://www.thehindu.com/news/national/coronav...,https://indianexpress.com/article/india/gujara...,Details awaited
1376,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
p_df.to_csv('patients_data.csv')